In [2]:
import sys
import torch
import torch.nn as nn
from torch.autograd import Variable

torch.manual_seed(777)
#            0   1   2    3   4
idx2char = ['h','i','e','l','o']
# Teach hihell -> ihello
x_data = [0,1,0,2,3,3] #hihell
one_hot_lookup = [[1, 0, 0, 0, 0],  # 0
                  [0, 1, 0, 0, 0],  # 1
                  [0, 0, 1, 0, 0],  # 2
                  [0, 0, 0, 1, 0],  # 3
                  [0, 0, 0, 0, 1]]  # 4
y_data = [1,0,2,3,3,4] #ihello
x_one_hot = [one_hot_lookup[x] for x in x_data]

In [3]:
inputs = Variable(torch.Tensor(x_one_hot))
labels = Variable(torch.LongTensor(y_data))

num_classes = 5
input_size = 5 #one-hot size
hidden_size = 5
batch_size = 1 #one sentence
sequence_length = 1
num_layers = 1

In [4]:
class Model(nn.Module):
    def __init__(self):
        super(Model,self).__init__()
        self.rnn = nn.RNN(input_size = input_size,
                         hidden_size = hidden_size,
                         batch_first = True)
    def forward(self, hidden, x):
        x = x.view(batch_size, sequence_length, input_size)
        
        out, hidden = self.rnn(x, hidden)
        return hidden, out.view(-1,num_classes)
    
    def init_hidden(self):
        return Variable(torch.zeros(num_layers, batch_size, hidden_size))

In [5]:
model = Model()
print(model)

Model(
  (rnn): RNN(5, 5, batch_first=True)
)


In [6]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.1)

In [7]:
for epoch in range(100):
    optimizer.zero_grad()
    loss = 0
    hidden = model.init_hidden()
    
    sys.stdout.write("predicted string : ")
    for inp, label in zip(inputs, labels):
        hidden, output = model(hidden, inp)
        val , idx = output.max(1)
        sys.stdout.write(idx2char[idx.data[0]])
        loss += criterion(output, torch.LongTensor([label]))
    print(", epoch: %d, loss : %1.3f" %(epoch + 1,loss))
    loss.backward()
    optimizer.step()
    
print("Learning finished !")

predicted string : llllll, epoch: 1, loss : 10.155
predicted string : llllll, epoch: 2, loss : 9.137
predicted string : llllll, epoch: 3, loss : 8.355
predicted string : llllll, epoch: 4, loss : 7.577
predicted string : llllll, epoch: 5, loss : 6.876
predicted string : lhelll, epoch: 6, loss : 6.327
predicted string : ihelll, epoch: 7, loss : 6.014
predicted string : ihelll, epoch: 8, loss : 5.787
predicted string : ihelll, epoch: 9, loss : 5.477
predicted string : ihelll, epoch: 10, loss : 5.274
predicted string : ihelll, epoch: 11, loss : 5.041
predicted string : ihello, epoch: 12, loss : 4.827
predicted string : ihello, epoch: 13, loss : 4.676
predicted string : ihello, epoch: 14, loss : 4.550
predicted string : ihello, epoch: 15, loss : 4.430
predicted string : ihello, epoch: 16, loss : 4.305
predicted string : ihello, epoch: 17, loss : 4.164
predicted string : ihelll, epoch: 18, loss : 4.003
predicted string : ihelll, epoch: 19, loss : 3.860
predicted string : ihelll, epoch: 20, l